<a href="https://colab.research.google.com/github/ManjuRama/FinMath/blob/main/FX_TFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

# Custom Dataset class for sequence generation
class FXDataset(Dataset):
    def __init__(self, X, y, seq_length):
        self.X = X
        self.y = y
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, idx):
        X_seq = self.X[idx:idx+self.seq_length]
        y_seq = self.y[idx+self.seq_length]
        return X_seq, y_seq

# Multi-head attention based model
class MultiHeadAttentionModel(nn.Module):
    def __init__(self, input_dim, seq_length, num_heads, hidden_dim):
        super(MultiHeadAttentionModel, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(input_dim * seq_length, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # Output is the predicted next hour volume
        )

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        attn_output = attn_output.flatten(start_dim=1)
        out = self.fc(attn_output)
        return out

# Prepare the data for training
def prepare_data(data, target_col, seq_length):
    """
    Prepares the data for training by scaling and creating sequences.
    - data: The input data containing the features and target
    - target_col: The column name of the target variable (next hour volume)
    - seq_length: The sequence length for the model
    """
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data.drop(columns=[target_col]))
    target = data[target_col].values

    return scaled_data, target, scaler

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch.float())
            loss = criterion(output.squeeze(), y_batch.float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}')

    return model

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch.float())
            predictions.append(output.squeeze().numpy())
            actuals.append(y_batch.numpy())

    return np.concatenate(predictions), np.concatenate(actuals)

# Load the dataset (replace with your actual file paths)
data_usdchf = pd.read_csv('USDCHF_data.csv', parse_dates=['timestamp'], index_col='timestamp')
data_gbpusd = pd.read_csv('GBPUSD_data.csv', parse_dates=['timestamp'], index_col='timestamp')
data_eurusd = pd.read_csv('EURUSD_data.csv', parse_dates=['timestamp'], index_col='timestamp')

# Set parameters
seq_length = 60  # Use 60 minutes as input sequence (1 hour)
num_heads = 4
hidden_dim = 128
num_epochs = 10
batch_size = 64

# Prepare and predict for each FX pair
for pair, data in [('USDCHF', data_usdchf), ('GBPUSD', data_gbpusd), ('EURUSD', data_eurusd)]:
    print(f'Predicting for {pair}...')

    # Prepare data for training
    X, y, scaler = prepare_data(data, target_col='volume_last_hour', seq_length=seq_length)

    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Create DataLoader for train and test sets
    train_dataset = FXDataset(X_train, y_train, seq_length)
    test_dataset = FXDataset(X_test, y_test, seq_length)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, loss function, and optimizer
    model = MultiHeadAttentionModel(input_dim=X_train.shape[1], seq_length=seq_length, num_heads=num_heads, hidden_dim=hidden_dim)
    criterion = nn.L1Loss()  # L1 loss for MAPE
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    model = train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

    # Evaluate the model
    predicted_volume, actual_volume = evaluate_model(model, test_loader)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(actual_volume, predicted_volume)
    print(f'{pair} - MAPE: {mape:.4f}')

    # Optional: plot actual vs predicted
    plt.figure(figsize=(10, 6))
    plt.plot(actual_volume, label='Actual Volume')
    plt.plot(predicted_volume, label='Predicted Volume', linestyle='--')
    plt.title(f'{pair} - Actual vs Predicted Next Hour Volume')
    plt.legend()
    plt.show()
